In [1]:
# !pip install -q -r requirements.txt
# pip install grandalf

In [64]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv("EnvVars.txt")

import os


In [42]:
prompt = PromptTemplate(
            template= "generate 5 interesting facts about {topic}",
            input_variables=['topic']
)

In [5]:
model = ChatOpenAI()

In [6]:
parser = StrOutputParser()

# Simple Chain

In [7]:
chain  = prompt | model | parser

In [8]:
result = chain.invoke({'topic': 'Rohit Sharma'})

In [22]:
print(result)

1. Rohit Sharma is the only player in the history of cricket to have scored three double centuries in One Day Internationals (ODIs).
2. He holds the record for the highest individual score in a One Day International match, scoring 264 runs against Sri Lanka in 2014.
3. Rohit Sharma is the first player to score four centuries in a single World Cup tournament, achieving this feat during the 2019 ICC Cricket World Cup.
4. He is the first captain to lead his team to five Indian Premier League (IPL) titles, captaining the Mumbai Indians to victory in 2013, 2015, 2017, 2019, and 2020.
5. Rohit Sharma is known for his elegant and stylish batting technique, often being compared to former Indian cricketer Virender Sehwag for his aggressive style of play.


In [28]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


# Sequential Chain

In [9]:
prompt1 = PromptTemplate(
            template= "generate a detialed report on topic {topic}",
            input_variables=['topic']
)

In [10]:
prompt2 = PromptTemplate(
            template= "generate a 5 pointer summary from the following text \n  {text}",
            input_variables=['text']
)

In [11]:
seq_chain = prompt1 | model | parser | prompt2 | model | parser

In [12]:
result = seq_chain.invoke({'topic': 'Recession in India'})

In [13]:
print(result)

1. A recession in India is characterized by a decrease in GDP, industrial production, increase in unemployment, and decrease in consumer spending.
2. The primary cause of the recession in India was the COVID-19 pandemic, which led to nationwide lockdowns and restrictions on economic activities.
3. The recession led to an increase in unemployment, decline in GDP, rise in poverty and inequality, and financial stress for many individuals and businesses.
4. Potential solutions to the recession in India include implementing stimulus packages, structural reforms, job creation programs, and diversifying the economy.
5. The government can take steps to help the economy recover, reduce unemployment, and build a more sustainable and resilient economy for the future.


In [14]:
seq_chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

# Parallel Chain

In [22]:
model1 = ChatOpenAI()
model2 = ChatOpenAI()#ChatAnthropic(model_name='claude-3')

In [23]:
prompt1 = PromptTemplate(
            template= "generate short and simple notes from the fololwing text \n {text}",
            input_variables=['text']
)

In [24]:
prompt2 = PromptTemplate(
            template= "generate 5 short questions from the fololwing text \n {text}",
            input_variables=['text']
)

In [25]:
prompt3 = PromptTemplate(
            template= "Merge the provided notes and quiz into a single document /n notes_> {notes} and \n quiz -> {quiz}",
            input_variables=['notes', 'quiz']
)

In [26]:
parallell_chain = RunnableParallel({
    
    'notes': prompt1 | model1 | parser ,
    'quiz': prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallell_chain | merge_chain

In [27]:
text = """
In statistics, linear regression is a model that estimates the linear relationship between a scalar response (dependent variable) and one or more explanatory variables (regressor or independent variable). A model with exactly one explanatory variable is a simple linear regression; a model with two or more explanatory variables is a multiple linear regression.[1] This term is distinct from multivariate linear regression, which predicts multiple correlated dependent variables rather than a single dependent variable.[2]

In linear regression, the relationships are modeled using linear predictor functions whose unknown model parameters are estimated from the data. Most commonly, the conditional mean of the response given the values of the explanatory variables (or predictors) is assumed to be an affine function of those values; less commonly, the conditional median or some other quantile is used. Like all forms of regression analysis, linear regression focuses on the conditional probability distribution of the response given the values of the predictors, rather than on the joint probability distribution of all of these variables, which is the domain of multivariate analysis.

Linear regression is also a type of machine learning algorithm, more specifically a supervised algorithm, that learns from the labelled datasets and maps the data points to the most optimized linear functions that can be used for prediction on new datasets.[3]

Linear regression was the first type of regression analysis to be studied rigorously, and to be used extensively in practical applications.[4] This is because models which depend linearly on their unknown parameters are easier to fit than models which are non-linearly related to their parameters and because the statistical properties of the resulting estimators are easier to determine.
"""

In [29]:
result = chain.invoke({'text': text})

In [30]:
print(result)

Linear regression is a supervised machine learning algorithm that estimates the relationship between a dependent variable and one or more independent variables. In simple linear regression, only one explanatory variable is used, while multiple linear regression involves two or more explanatory variables.

The relationships in linear regression are modeled by fitting a linear equation to the data, which represents the relationship between the independent variables and the dependent variable. The goal is to find the best-fitting line that minimizes the sum of the squared differences between the observed and predicted values.

Linear regression focuses on the conditional probability distribution of the response given the predictors, making it a valuable tool for predicting the outcome based on input variables. On the other hand, multivariate linear regression involves predicting the dependent variable using multiple input variables.

Linear regression was the first type of regression anal

In [31]:
chain.get_graph().print_ascii()

            +---------------------------+            
            | Parallel<notes,quiz>Input |            
            +---------------------------+            
                 **               **                 
              ***                   ***              
            **                         **            
+----------------+                +----------------+ 
| PromptTemplate |                | PromptTemplate | 
+----------------+                +----------------+ 
          *                               *          
          *                               *          
          *                               *          
  +------------+                    +------------+   
  | ChatOpenAI |                    | ChatOpenAI |   
  +------------+                    +------------+   
          *                               *          
          *                               *          
          *                               *          
+-----------------+         

# Conditional Chain

In [51]:
class Feedback(BaseModel):
    sentiment : Literal['positive', 'negative'] = Field(description='give the sentiment of the feedback')

In [52]:
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [53]:
prompt1 = PromptTemplate(
            template= "classify the sentiment of the following feedback into positive or negative\n {feedback} \n {format_instruction}",
            input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

In [54]:
classifier_chain = prompt1 | model | parser2

In [58]:
sentiment = classifier_chain.invoke({'feedback':'the product in very bad but decent charging'})

In [60]:
prompt2 = PromptTemplate(
            template= "write an appropriate response to this positive feedback \n {feedback}",
            input_variables=['feedback']
)

In [61]:
prompt3 = PromptTemplate(
            template= "write an appropriate response to this negative feedback \n {feedback}",
            input_variables=['feedback']
)

In [69]:
branch_chain = RunnableBranch(
         (lambda x : x.sentiment == 'positive', prompt2 | model | parser ),
         (lambda x : x.sentiment == 'negative', prompt3 | model | parser),
         RunnableLambda(lambda x : "Could not find sentiment")
    
)

In [70]:
final_chain = classifier_chain | branch_chain

In [75]:
final_chain.invoke('THis is a veryexcellent phone')

"Thank you so much for the positive feedback! I'm glad to hear that you are satisfied with our service. It was a pleasure assisting you and we look forward to serving you again in the future."

In [76]:
final_chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
     +------------+      
     | ChatOpenAI |      
     +------------+      
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
